# **DEX,AMM with Pytezos**
In this lesson you will learn:
- How decentralized exchanges work
- What is AMMM
- Learn how to calculate the rate for pairs of tokens, which you will need in your bot to compare rates on different exchanges

First, install the necessary libraries for this lesson, and then move on to the theory

In [ ]:
%%capture
## %%capture is a Colab built-in function to suppress the output of that particular cell whether it uses a command-line code or some python code

## Pytezos dependencies
!apt install libsodium-dev libsecp256k1-dev libgmp-dev
!pip install pytezos

# **What is a DEX?**

Unlike traditional orderbook-based exchanges, where buyers and sellers cross the bid-ask spread to execute trades and determine asset prices, Automated Market Makers (AMM) are a type of decentralized exchange composed of “smart contract-liquidity pools that hold (at least) two crypto assets in reserve and allow anyone to deposit tokens of one type and thereby to withdraw tokens of the other type 

Examples of Tezos DEXes:
 - https://quipuswap.com/swap
 - https://spicyswap.xyz/
 - https://www.plentydefi.com/swap
 - https://aliens.farm/exchange

let's connect to one of these contracts from the Quipuswap exchange

In [ ]:
from pytezos import pytezos
# Tezos в Quipu
TeztoQ = pytezos.using('mainnet').contract('KT1X3zxdTzPB9DgVzA3ad6dgZe9JEamoaeRy')

Each pool is defined by a smart contract that includes a few functions to enable swapping tokens, adding liquidity, and more. At its core, each pool uses the function $x * y = k$ to maintain a curve along which trades can happen.

Usually dex contracts include a description of the current liquidity, let's look at an example. This contract has token_pool and tez_pool which are x and y in the equation above/

In [ ]:
TeztoQ.storage()

{'dex_lambdas': 12073,
 'metadata': 12074,
 'storage': {'baker_validator': 'KT1DLRZD5XTQYydXzGuPhXUx8TdZR2WEmMGS',
  'current_candidate': 'tz1VQnqCCqX4K5sP3FNkVSNKTdCAMJDd3E1n',
  'current_delegated': 'tz1UNkf4sWzC5vsKP7FnejSeJsXdc6z8Kzk4',
  'last_update_time': 1644760274,
  'last_veto': 1630028166,
  'ledger': 12075,
  'period_finish': 1645536246,
  'reward': 54474664,
  'reward_paid': 403830698,
  'reward_per_sec': 57934389660493827,
  'reward_per_share': 15187608701188,
  'tez_pool': 42450770044,
  'token_address': 'KT193D4vozYnhGJQVtw7CoxxqphqUEEwK6Vb',
  'token_id': 0,
  'token_pool': 72951595081,
  'total_reward': 671460603,
  'total_supply': 46475786541,
  'total_votes': 6471452357,
  'user_rewards': 12076,
  'veto': 155343,
  'vetos': 12077,
  'voters': 12078,
  'votes': 12079},
 'token_lambdas': 12080}

Let's take these variables and use them to calculate the exchange rate

In [ ]:
tez_pool = TeztoQ.storage()['storage']['tez_pool']
tez_pool

42450770044

In [ ]:
token_pool = TeztoQ.storage()['storage']['token_pool']
token_pool

72951595081

### Token prices

Considering that the equation  $x * y = k$ must hold, when someone executes a trade, we get $(x + Δx) · (y + Δy) = k$. It can then be easily shown that $Δy = (k/x + Δx) – y$.

**Important**: Tez prices are considered in microthesis, and token prices are in decimals from contracts

In [ ]:
#1 tez = 1,000,000 micro tez
def tzToMutez(tezAmount):
    return tezAmount*1000000

### Fees

The trading fee is set at 0.3%,out of which is added to the liquidity pools as rewards for the liquidity providers

In [ ]:
def TeztoQoutput(tezAmount):
    mutezAmount = tzToMutez(tezAmount)
    
    tezInWithFee = mutezAmount*997
    
    numerator = tezInWithFee*token_pool
    
    denominator = tez_pool * 1000 + tezInWithFee
    
    tokensOut = numerator/denominator

    return tokensOut/(10*10*10*10*10*10)


## Task1 
As you can see we we divide the tokens by 10 to the power of 6, as it was mentioned before, the calculations are carried out in decimals of tokens. As an example, we transfer Tetz to Quipu, find the Quipu token in https://better-call.dev/ and make sure that you need to divide by this number

In [ ]:
TeztoQoutput(1)

1.7133030109293605

Now let's calculate Exchange rate

In [ ]:
#Exchange rate
def ex_rate_tq(amount):
    return (amount)/TeztoQoutput(amount)


In [ ]:
ex_rate_tq(100)



0.5850249814729209

# What is Slippage Tolerance?
Slippage Tolerance is the pricing difference between the price at the confirmation time and the actual price of the transaction users are willing to accept when swapping on AMMs. Slippage tolerance is set as a percentage of the total swap value.
In the case of volatile markets or low-liquidity pools, users can complete the transactions faster and avoid failures by accepting a higher slippage and slippage tolerance percentage.

In [ ]:
#Slippage tolerance
s_t=0.005

#Minimum received

def minimum_received(s_t,amount):
  multiplier = 1 - s_t
  return TeztoQoutput(amount)*multiplier

minimum_received(s_t,1)

1.7047364958747138

Let's get everything calculated

In [ ]:
# Result Tez to Quipu
tez_amount = 1
print("Input {} XTZ".format(tez_amount))
print("Output {} QUIPU".format(TeztoQoutput(tez_amount)))
print("Exchange rate 1 Quipuswap Governance Token = {} Tezos".format(round(ex_rate_tq(tez_amount),6)))

Input 1 XTZ
Output 1.7133030109293605 QUIPU
Exchange rate 1 Quipuswap Governance Token = 0.583668 Tezos


## Task2 
Compare the received values with the values at https://v1.quipuswap.com/swap
Note that the built-in rounding in Python and the rounding in the Bignumber.js library used by Quipuswap are different.

## Task3 

Write a conversion formula from token A to token B, the formula can be peeped here: https://github.com/madfish-solutions/quipuswap-webapp/blob/6cb9c935edf3346207daded889281811060fae2c/src/core/estimates.ts

